In [1]:
import numpy as np
import pandas as pd
import math
data=pd.read_csv('data0.csv')
data.head()

,Unnamed: 0,member_id,sku_id,num
0,0,250,742,2
1,1,250,3371,1
2,2,283,1198,1
3,3,279,1102,1
4,4,279,5721,1


In [2]:
data.drop(columns='Unnamed: 0',inplace=True)
data.head()

,member_id,sku_id,num
0,250,742,2
1,250,3371,1
2,283,1198,1
3,279,1102,1
4,279,5721,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59634 entries, 0 to 59633
Data columns (total 3 columns):
member_id    59634 non-null int64
sku_id       59634 non-null int64
num          59634 non-null int64
dtypes: int64(3)
memory usage: 1.4 MB


将该数据转化二维

In [4]:
data_pivot = data.pivot(index='member_id',columns='sku_id',values='num')
data_pivot.head()

sku_id,536,537,538,539,540,552,553,554,555,556,...,18183,18184,18185,18186,18187,18188,18189,18190,18191,18192
member_id,,,,,,,,,,,,,,,,,,,,,
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data_matrix=data_pivot.fillna(0).values
data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
sku_similar=cosine_similarity(data_matrix.T) #计算data_matrix的转置矩阵的余弦相似度
#构建预测函数
def predict(similar):
    memb_cnt = data_matrix.shape[0]  #会员数14977
    sku_cnt = data_matrix.shape[1]  #商品数3824
    pred = np.zeros((memb_cnt,sku_cnt))
    for mid in range(memb_cnt):
        for sid in range(sku_cnt):
            if data_matrix[mid,sid]==0:
                pred[mid,sid]=Recommendation_s(mid,sid,similar)
    return pred

注： for mid in range(memb_cnt):

In [7]:
#构建一个物品的推荐
def Recommendation_s(mid,sid,similar,k=10):
    score=0
    weight=0
    mid_action=data_matrix[mid,:]     #会员mid对所有商品的购买次数
    sid_action=data_matrix[:,sid]    #商品sid得到的所有会员的购买次数
    sid_similar=similar[sid,:]      #商品sid对所有商品的相似度
    sim_indexs=np.argsort(sid_similar)[-(k+1):-1]   #最相似的K个商品除了index（除了自己）
    sid_s_mean=np.sum(sid_action)/sid_action[sid_action!=0].size
    for j in sim_indexs:
        if mid_action[j]!=0:
            sid_j_action=data_matrix[:,j]
            sid_j_mean=np.sum(sid_j_action)/sid_j_action[sid_j_action!=0].size
            score+=sid_similar[j]*(mid_action[j]-sid_j_mean)
            weight+=abs(sid_similar[j])
    if weight==0:
        return 0
    else:
        return sid_s_mean+score/float(weight)

In [8]:
memb_prediction=predict(sku_similar)
memb_prediction

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
def get_recom(prediction,k=5):
    recom_df=pd.DataFrame(prediction,columns=data_pivot.columns,index=data_pivot.index)
    recom_df=recom_df.stack().reset_index()
    recom_df.rename(columns={0:"推荐指数"},inplace=True)
    grouped=recom_df.groupby('member_id')
    topk=grouped.apply(get_topk,k=k)
    topk=topk.drop(['member_id'],axis=1)
    topk.index=topk.index.droplevel(1)
    topk.reset_index(inplace=True)
    return topk
def get_topk(group,k):
    return group.sort_values('推荐指数',ascending=False)[:k]
recom_df=pd.DataFrame(memb_prediction,columns=data_pivot.columns,index=data_pivot.index)
recom_df.head()

sku_id,536,537,538,539,540,552,553,554,555,556,...,18183,18184,18185,18186,18187,18188,18189,18190,18191,18192
member_id,,,,,,,,,,,,,,,,,,,,,
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
memb_recom=get_recom(memb_prediction,2)
memb_recom

,member_id,sku_id,推荐指数
0,243,4534,1.000000
1,243,973,1.000000
2,244,5777,1.000000
3,244,5556,1.000000
4,250,3266,1.000000
...,...,...,...
29949,52184,4336,1.000000
29950,52185,7858,1.500000
29951,52185,1414,1.000000
29952,52186,3049,1.465517


In [11]:
memb_recom.推荐指数.value_counts()

1.000000    11899
1.483651      576
1.250000      430
1.485882      330
1.076923      307
            ...  
1.931592        1
1.009175        1
3.983480        1
2.009197        1
1.032664        1
Name: 推荐指数, Length: 936, dtype: int64